### Import libraries

In [ ]:
import warnings

warnings.simplefilter(action="ignore")

In [ ]:
import matplotlib.pyplot as plt

%matplotlib qt
import mne
import numpy as np
import pandas as pd
import os
from itertools import compress

### Define ROIs

In [ ]:
leftMotor = [
    "S1_D1 hbo",
    "S1_D2 hbo",
    "S1_D3 hbo",
    "S2_D1 hbo",
    "S2_D3 hbo",
    "S2_D4 hbo",
    "S3_D2 hbo",
    "S3_D3 hbo",
    "S4_D3 hbo",
    "S4_D4 hbo",
]
rightMotor = [
    "S5_D5 hbo",
    "S5_D6 hbo",
    "S5_D7 hbo",
    "S6_D5 hbo",
    "S6_D7 hbo",
    "S6_D8 hbo",
    "S7_D6 hbo",
    "S7_D7 hbo",
    "S8_D7 hbo",
    "S8_D8 hbo",
]
leftFrontal = [
    "S9_D9 hbo",
    "S9_D10 hbo",
    "S10_D9 hbo",
    "S10_D11 hbo",
    "S11_D9 hbo",
    "S11_D11 hbo",
    "S11_D12 hbo",
    "S12_D10 hbo",
    "S12_D13 hbo",
]
rightFrontal = [
    "S13_D11 hbo",
    "S13_D14 hbo",
    "S14_D12 hbo",
    "S14_D14 hbo",
    "S14_D15 hbo",
    "S15_D13 hbo",
    "S15_D15 hbo",
    "S16_D14 hbo",
    "S16_D15 hbo",
]
medialFrontal = ["S12_D12 hbo", "S13_D12 hbo"]

### Load results output structure
(1) HbO signal mean or median

(2) HbO signal variance

In [ ]:
fNIRSresHbO = pd.read_excel("RestingStateResults_June2023.xlsx")
fNIRSvarHbO = pd.read_excel("RestingStateVar_June2023.xlsx")
fNIRSrawHbO = pd.read_excel("RestingStateRaw_June2023.xlsx")

### Load sample info

In [ ]:
sid = input("SID:")
condition = input("Bedingung (HIIT, nonHIIT):")
session = input("Messung (base, r1, r2, r3):")
dataDir = input("Datenverzeichnis:")
parameter = input("Parameter (mean, median):")

### Load raw fNIRS intensities

In [ ]:
raw_intensity = mne.io.read_raw_nirx(
    os.getcwd() + "/" + sid + "/" + condition + "/" + dataDir, verbose=True
)
raw_intensity.load_data()

### Set block duration and show raw data (only for quality assessment)

In [ ]:
# raw_intensity.annotations.delete([1,2,3,4])
raw_intensity.annotations.set_durations(240)
raw_intensity.plot(duration=300)

### Converting from raw intensity to optical density
The raw intensity values are then converted to optical density.

In [ ]:
raw_od = mne.preprocessing.nirs.optical_density(raw_intensity)
raw_od.plot(duration=300)

### Evaluating the quality of the data

At this stage we can quantify the quality of the coupling between the scalp and the optodes using the scalp coupling index (sci). The sci ranges from 0 (no optical transmission along an optical path) to 1 (full optical transmission). This method looks for the presence of a prominent synchronous signal (the heart rate) in the frequency range of cardiac signals across both photodetected signals.

In [ ]:
sci = mne.preprocessing.nirs.scalp_coupling_index(raw_od)
plt.figure()
plt.hist(sci)
plt.xlabel("Scalp Coupling Index")
plt.ylabel("Count")
plt.show()

### Labelling all channes with an sci < 0.5 as bad

In [ ]:
raw_od.info["bads"] = raw_od.info["bads"] + list(compress(raw_od.ch_names, sci < 0.5))
print(f'Bad channels: {raw_od.info["bads"]}')

### Artifact correction
Apply temporal derivative distribution repair
This approach corrects baseline shift and spike artifacts without the need for any user-supplied parameters
according to:

Frank A Fishburn, Ruth S Ludlum, Chandan J Vaidya, and Andrei V Medvedev. Temporal derivative distribution repair (tddr): a motion correction method for fNIRS. NeuroImage, 184:171–179, 2019. doi:10.1016/j.neuroimage.2018.09.025.

In [ ]:
corrected_tddr = mne.preprocessing.nirs.temporal_derivative_distribution_repair(raw_od)
corrected_tddr.plot(duration=300)

### Converting from optical density to haemoglobin
Convert the optical density data to haemoglobin concentration using the modified Beer-Lambert law.

In [ ]:
raw_haemo = mne.preprocessing.nirs.beer_lambert_law(corrected_tddr, ppf=0.1)
raw_haemo.plot(duration=300)

### Filtering

Removing heart rate from signal. The haemodynamic response has (in our case) frequency content predominantly below about 0.05 Hz. An increase in activity around 1 Hz can be seen in the data that is due to the person’s heart beat and is unwanted. So we use a low pass filter to remove this. A high pass filter is also included to remove slow drifts in the data.

In [ ]:
raw_haemo_unfiltered = raw_haemo.copy()
raw_haemo.filter(0.05, 0.5, h_trans_bandwidth=0.2, l_trans_bandwidth=0.02)
for when, _raw in dict(Before=raw_haemo_unfiltered, After=raw_haemo).items():
    fig = _raw.compute_psd().plot(average=True, picks="data", exclude="bads")
    fig.suptitle(f"{when} filtering", weight="bold", size="x-large")
    fig.subplots_adjust(top=0.88)

In [ ]:
raw_haemo.plot(duration=300)

### Extract HbO values within marked segment

Define segment data points

In [ ]:
start_resting = 30 * 5
end_resting = 270 * 5

Get signal within ROIs

In [ ]:
oxy_LeftMotor = raw_haemo.get_data(leftMotor, start_resting, end_resting)
oxy_RightMotor = raw_haemo.get_data(rightMotor, start_resting, end_resting)
oxy_LeftFrontal = raw_haemo.get_data(leftFrontal, start_resting, end_resting)
oxy_RightFrontal = raw_haemo.get_data(rightFrontal, start_resting, end_resting)
oxy_MedialFrontal = raw_haemo.get_data(medialFrontal, start_resting, end_resting)

Log parameter raw values (RestingStateRaw_June2023.xlsx) 

and time series variances (RestingStateVar_June2023.xlsx)

In [ ]:
if parameter == "median":
    fNIRSrawHbO[condition + "_FM" + session][fNIRSrawHbO.SID == sid] = np.median(
        oxy_MedialFrontal
    )
    fNIRSrawHbO[condition + "_FL" + session][fNIRSrawHbO.SID == sid] = np.median(
        oxy_LeftFrontal
    )
    fNIRSrawHbO[condition + "_FR" + session][fNIRSrawHbO.SID == sid] = np.median(
        oxy_RightFrontal
    )
    fNIRSrawHbO[condition + "_ML" + session][fNIRSrawHbO.SID == sid] = np.median(
        oxy_LeftMotor
    )
    fNIRSrawHbO[condition + "_MR" + session][fNIRSrawHbO.SID == sid] = np.median(
        oxy_RightMotor
    )
if parameter == "mean":
    fNIRSrawHbO[condition + "_FM" + session][fNIRSrawHbO.SID == sid] = np.mean(
        oxy_MedialFrontal
    )
    fNIRSrawHbO[condition + "_FL" + session][fNIRSrawHbO.SID == sid] = np.mean(
        oxy_LeftFrontal
    )
    fNIRSrawHbO[condition + "_FR" + session][fNIRSrawHbO.SID == sid] = np.mean(
        oxy_RightFrontal
    )
    fNIRSrawHbO[condition + "_ML" + session][fNIRSrawHbO.SID == sid] = np.mean(
        oxy_LeftMotor
    )
    fNIRSrawHbO[condition + "_MR" + session][fNIRSrawHbO.SID == sid] = np.mean(
        oxy_RightMotor
    )

fNIRSrawHbO.to_excel("RestingStateRaw_June2023.xlsx")


fNIRSvarHbO[condition + "_FM" + session][fNIRSvarHbO.SID == sid] = np.var(
    oxy_MedialFrontal
)
fNIRSvarHbO[condition + "_FL" + session][fNIRSvarHbO.SID == sid] = np.var(
    oxy_LeftFrontal
)
fNIRSvarHbO[condition + "_FR" + session][fNIRSvarHbO.SID == sid] = np.var(
    oxy_RightFrontal
)
fNIRSvarHbO[condition + "_ML" + session][fNIRSvarHbO.SID == sid] = np.var(oxy_LeftMotor)
fNIRSvarHbO[condition + "_MR" + session][fNIRSvarHbO.SID == sid] = np.var(
    oxy_RightMotor
)

fNIRSvarHbO.to_excel("RestingStateVar_June2023.xlsx")

Data propcessing according to Ishikawa et al.: Subtract channel-wise min values

and log parameter values (RestingStateResults_June2023.xlsx)

In [ ]:
for ii in range(0, len(oxy_LeftMotor)):
    oxy_LeftMotor[ii, :] = oxy_LeftMotor[ii, :] - np.min(oxy_LeftMotor[ii, :])
for ii in range(0, len(oxy_RightMotor)):
    oxy_RightMotor[ii, :] = oxy_RightMotor[ii, :] - np.min(oxy_RightMotor[ii, :])
for ii in range(0, len(oxy_LeftFrontal)):
    oxy_LeftFrontal[ii, :] = oxy_LeftFrontal[ii, :] - np.min(oxy_LeftFrontal[ii, :])
for ii in range(0, len(oxy_RightFrontal)):
    oxy_RightFrontal[ii, :] = oxy_RightFrontal[ii, :] - np.min(oxy_RightFrontal[ii, :])
for ii in range(0, len(oxy_MedialFrontal)):
    oxy_MedialFrontal[ii, :] = oxy_MedialFrontal[ii, :] - np.min(
        oxy_MedialFrontal[ii, :]
    )

In [ ]:
if parameter == "median":
    fNIRSresHbO[condition + "_FM" + session][fNIRSresHbO.SID == sid] = np.median(
        oxy_MedialFrontal
    )
    fNIRSresHbO[condition + "_FL" + session][fNIRSresHbO.SID == sid] = np.median(
        oxy_LeftFrontal
    )
    fNIRSresHbO[condition + "_FR" + session][fNIRSresHbO.SID == sid] = np.median(
        oxy_RightFrontal
    )
    fNIRSresHbO[condition + "_ML" + session][fNIRSresHbO.SID == sid] = np.median(
        oxy_LeftMotor
    )
    fNIRSresHbO[condition + "_MR" + session][fNIRSresHbO.SID == sid] = np.median(
        oxy_RightMotor
    )
if parameter == "mean":
    fNIRSresHbO[condition + "_FM" + session][fNIRSresHbO.SID == sid] = np.mean(
        oxy_MedialFrontal
    )
    fNIRSresHbO[condition + "_FL" + session][fNIRSresHbO.SID == sid] = np.mean(
        oxy_LeftFrontal
    )
    fNIRSresHbO[condition + "_FR" + session][fNIRSresHbO.SID == sid] = np.mean(
        oxy_RightFrontal
    )
    fNIRSresHbO[condition + "_ML" + session][fNIRSresHbO.SID == sid] = np.mean(
        oxy_LeftMotor
    )
    fNIRSresHbO[condition + "_MR" + session][fNIRSresHbO.SID == sid] = np.mean(
        oxy_RightMotor
    )

fNIRSresHbO.to_excel("RestingStateResults_June2023.xlsx")